In [58]:
from pathlib import Path

import numpy as np
import pandas as pd
import json

import starfile
import napari
import mrcfile

In [59]:
viewer = napari.Viewer(ndisplay=3)

In [84]:
eman2_directory = Path("/mnt/scratch/ribosomes/kas_k44a/eman2/")

eman2_particle_lists = sorted(eman2_directory.glob("info/corrected_iter30-ts_*_b10_rec_corrected_info.json"))

eman2_particle_lists[:5]

[PosixPath('/mnt/scratch/ribosomes/kas_k44a/eman2/info/corrected_iter30-ts_004_b10_rec_corrected_info.json')]

In [87]:
ts_index = 0

ts_name = eman2_particle_lists[ts_index].name.split("-")[1][:6]
print(ts_name)

particle_data = json.load(open(eman2_particle_lists[ts_index]))

box_class_map = {}
for key in particle_data["class_list"].keys():
    box_class_map[str(key)] = particle_data["class_list"][key]["name"]

particle_df = pd.DataFrame(particle_data["boxes_3d"], columns=["x","y","z","","","box_class"]).drop(columns="")
particle_df["box_class"] = particle_df["box_class"].astype(str).map(box_class_map)

particle_df.head()


ts_004


,x,y,z,box_class
0,758,448,139,box20_mass1000
1,632,34,124,box20_mass1000
2,622,562,185,box20_mass1000
3,580,716,287,box20_mass1000
4,172,1132,353,box20_mass1000


In [88]:
tomogram_path = [tomogram for tomogram in eman2_directory.glob(f"../isonet/corrected_iter30/{ts_name}*.mrc")]
segmentation_path = [seg for seg in eman2_directory.glob(f"segmentations/{ts_name}*.mrc")]
if len(list(segmentation_path)) == 0:
    #error: no file found
    print("no file found")
    pass
elif len(list(segmentation_path)) > 1:
    #error: multiple files found
    print("multiple files found")
    pass

tomogram_layer = viewer.add_image(
    mrcfile.read(tomogram_path[0]),
    name="tomogram",
    blending="translucent",
    depiction="plane",
    visible=False,
)
segementation_layer = viewer.add_image(
    mrcfile.read(segmentation_path[0]),
    name="segmentation",
    blending="translucent",
    colormap="gist_earth",
    visible=False,
)

In [90]:
particle_data = json.load(open(eman2_particle_lists[ts_index]))

box_class_map = {}
for key in particle_data["class_list"].keys():
    box_class_map[str(key)] = particle_data["class_list"][key]["name"]

particle_df = pd.DataFrame(particle_data["boxes_3d"], columns=["x","y","z","","","box_class"]).drop(columns="")
particle_df["box_class"] = particle_df["box_class"].astype(str).map(box_class_map)

particle_df["x"] = particle_df["x"]# - 10
colors = ["red", "green", "blue", "yellow", "magenta", "cyan", "white", "gray", "black",]
i = 0
for box_class in particle_df["box_class"].unique():
    viewer.add_points(
        particle_df[particle_df["box_class"] == box_class][["z","y","x"]],
        name=box_class,
        blending="translucent",
        edge_color = "red",
        edge_width = 0.2,
        visible=False,
    )
    i += 1

In [47]:
df.columns = ["rlnCoordinateX", "rlnCoordinateY", "rlnCoordinateZ"]
rln_angles = ["rlnAngleRot", "rlnAngleTilt", "rlnAnglePsi"]
for angle in rln_angles:
    random_angles = np.random.randint(0, 179, size=len(df.index))
    df[angle] = random_angles